In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import matplotlib as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from scipy.ndimage import shift
from IPython.display import clear_output
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
import seaborn
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import PolynomialFeatures

In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [3]:
train_data = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/train.csv"))
test_data = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/test.csv"))
sample_submission = pd.read_csv(Path("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv"))
train_data = train_data.drop("Id",axis=1)
test_data = test_data.drop("Id",axis=1)

In [4]:
X_train_data = train_data.drop("SalePrice",axis=1)
y_train_data = train_data["SalePrice"]

In [5]:
X_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1459 non-null   int64  
 1   MSZoning       1455 non-null   object 
 2   LotFrontage    1232 non-null   float64
 3   LotArea        1459 non-null   int64  
 4   Street         1459 non-null   object 
 5   Alley          107 non-null    object 
 6   LotShape       1459 non-null   object 
 7   LandContour    1459 non-null   object 
 8   Utilities      1457 non-null   object 
 9   LotConfig      1459 non-null   object 
 10  LandSlope      1459 non-null   object 
 11  Neighborhood   1459 non-null   object 
 12  Condition1     1459 non-null   object 
 13  Condition2     1459 non-null   object 
 14  BldgType       1459 non-null   object 
 15  HouseStyle     1459 non-null   object 
 16  OverallQual    1459 non-null   int64  
 17  OverallCond    1459 non-null   int64  
 18  YearBuil

In [7]:
X_train_data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


**Prepairing the data :**

In [8]:
imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(pd.concat([X_train_data,test_data]))

SimpleImputer(strategy='most_frequent')

In [9]:
c = X_train_data.columns
i_train = X_train_data.index
i_test = test_data.index
X_train_data = pd.DataFrame(imputer.transform(X_train_data),columns = c,index = i_train)
test_data = pd.DataFrame(imputer.transform(test_data),columns = c,index=i_test)

In [10]:
 str_cat = [ "MSZoning"      ,
  "Street"        ,
  "Alley"         ,
  "LotShape"      ,
  "LandContour"   ,
  "Utilities"     ,
  "LotConfig"     ,
  "LandSlope"     ,
  "Neighborhood"  ,
  "Condition1"    ,
  "Condition2"    ,
  "BldgType"      ,
  "HouseStyle"    ,
  "RoofStyle"     ,
  "RoofMatl"      ,
  "Exterior1st"   ,
  "Exterior2nd"   ,
  "MasVnrType"    ,
  "ExterQual"     ,
  "ExterCond"     ,
  "Foundation"    ,
  "BsmtQual"      ,
  "BsmtCond"      ,
  "BsmtExposure"  ,
  "BsmtFinType1"  ,
  "BsmtFinType2"  ,
  "Heating"       ,
  "HeatingQC"     ,
  "CentralAir"    ,
  "Electrical"    ,
  "KitchenQual"   ,
  "Functional"    ,
  "FireplaceQu"   ,
  "GarageType"    ,
  "GarageFinish"  ,
  "GarageQual"    ,
  "GarageCond"    ,
  "PavedDrive"    ,
  "PoolQC"        ,
  "Fence"         ,
  "MiscFeature"   ,
  "SaleCondition" ,
  "SaleType"      ]

In [11]:
def OneHotE(dataframe,cat_col,cat_encoder):
    dataframe_num = dataframe.drop(cat_col,axis=1)
    dataframe_cat = dataframe[cat_col]
    dataframe_cat = cat_encoder.transform(dataframe_cat)
    dataframe_cat = pd.DataFrame(dataframe_cat.toarray(),columns=cat_encoder.get_feature_names_out(),index=dataframe.index)
    dataframe = pd.concat([dataframe_num,dataframe_cat],axis=1)    
    return dataframe

In [12]:
cat_encoder = OneHotEncoder()

In [13]:
cat_encoder.fit(pd.concat([X_train_data[str_cat],test_data[str_cat]]))

OneHotEncoder()

In [14]:
X_train_data = OneHotE(X_train_data,str_cat,cat_encoder)
test_data = OneHotE(test_data,str_cat,cat_encoder)

In [15]:
scaler = StandardScaler()
for column in X_train_data.columns:
    X_train_data[column] = scaler.fit_transform(np.array(X_train_data[column]).reshape(-1,1))

In [16]:
poly_features = PolynomialFeatures(degree=2,include_bias=False)
X_train_data_poly =  poly_features.fit_transform(X_train_data)

In [17]:
X_train_data_poly = pd.DataFrame(X_train_data_poly,index = X_train_data.index , columns = poly_features.get_feature_names_out(X_train_data.columns))

In [18]:
test_data_poly =  poly_features.fit_transform(test_data)
test_data_poly = pd.DataFrame(test_data_poly,index = test_data.index , columns = poly_features.get_feature_names_out(test_data.columns))

In [19]:
test_data_poly.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw^2,SaleType_ConLw SaleType_New,SaleType_ConLw SaleType_Oth,SaleType_ConLw SaleType_WD,SaleType_New^2,SaleType_New SaleType_Oth,SaleType_New SaleType_WD,SaleType_Oth^2,SaleType_Oth SaleType_WD,SaleType_WD^2
0,20.0,80.0,11622.0,5.0,6.0,1961.0,1961.0,0.0,468.0,144.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,20.0,81.0,14267.0,6.0,6.0,1958.0,1958.0,108.0,923.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,60.0,74.0,13830.0,5.0,5.0,1997.0,1998.0,0.0,791.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,60.0,78.0,9978.0,6.0,6.0,1998.0,1998.0,20.0,602.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,120.0,43.0,5005.0,8.0,5.0,1992.0,1992.0,0.0,263.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


**Training a model:**

In [20]:
pipeline = Pipeline([("classifier",LinearRegression())])

In [21]:
param_grid = [{"classifier__fit_intercept":[True,False]}] 

In [22]:
grid_search = GridSearchCV(pipeline,param_grid,cv=3,scoring="r2")

In [23]:
grid_search.fit(X_train_data_poly,y_train_data)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('classifier', LinearRegression())]),
             param_grid=[{'classifier__fit_intercept': [True, False]}],
             scoring='r2')

In [24]:
grid_search.best_params_


{'classifier__fit_intercept': True}

In [25]:
cv_res = pd.DataFrame(grid_search.cv_results_)
cv_res.sort_values(by="mean_test_score", ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__fit_intercept,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,9.383453,1.232223,0.775046,0.006112,True,{'classifier__fit_intercept': True},0.759714,0.746709,0.645182,0.717202,0.051202,1
1,9.056386,1.102167,0.758638,0.002992,False,{'classifier__fit_intercept': False},0.694994,0.679981,0.407341,0.594105,0.132205,2


**Prediction :**

In [26]:
final_evaluation = grid_search.predict(test_data_poly)

In [27]:
d = {'Id':np.arange(len(final_evaluation)) + 1461,'SalePrice':final_evaluation}
df = pd.DataFrame(data=d)
df

,Id,SalePrice
0,1461,4.361029e+10
1,1462,7.724727e+10
2,1463,6.854862e+10
3,1464,4.102125e+10
4,1465,1.465509e+10
...,...,...
1454,2915,8.438898e+09
1455,2916,9.134880e+09
1456,2917,1.298461e+11
1457,2918,3.489404e+10


In [28]:
df.to_csv('out.csv',index=False)